In [1]:
import tarfile
import pandas as pd
import datetime as dt
import os
import glob
import subprocess
import os.path
from os import path
import json
from pathlib import Path

In [2]:
Gantry_2020_03_03_tar = tarfile.open('2020-03-03_clean.tar.gz')
Gantry_2020_03_03_tar.extractall('2020_03_03')
Gantry_2020_03_03_tar.close()

In [3]:
class Thermal:
    def get_tar(season, date, hour, minute):
        command = f'iget -rKTPf -N 0 /iplant/home/shared/terraref/ua-mac/level_1/{season}/EnvironmentLogger/{date}_clean.tar.gz'
        subprocess.call(command, shell = True)
        command = f'tar -xvf {date}_clean.tar.gz'
        subprocess.call(command, shell = True)

## Retrieve csv data and organize/clean up
    def retrieve_data(season, date, hour, minute):
        tar_data = Thermal.get_tar(season, date, hour, minute)
        data = pd.read_csv(f'./{date}_clean.csv')
        data['Time'] = pd.to_datetime(data['Time'])
        #data['date'] = data.Time.dt.date
        #data['time'] = data.Time.dt.time
        data_clean = data[['Time', 'Sun Direction', 'Temperature', 
                                      'Photosynthetically active radiation']]
        #index_data = data_clean.set_index('time')
        return data_clean

## Select Time Specified
    def split_time(season, date, hour, minute):
        index_data = Thermal.retrieve_data(season, date, hour, minute)
        specific_time = index_data[index_data.index == dt.time (hour, minute)]
        return specific_time

In [4]:
class JSON:
    def get_tar(season, date):
        command = f'iget -rKTPf -N 0 /iplant/home/shared/terraref/ua-mac/raw_tars/{season}/flirIrCamera/flirIrCamera-{date}.tar'
        subprocess.call(command, shell = True)
        command = f'tar -xvf flirIrCamera-{date}.tar'
        subprocess.call(command, shell = True)
    
    def pathlist(season, date):
        json_data = JSON.get_tar(season, date)
        pathlist = Path(f"./flirIrCamera/{date}/").glob('**/*.json')
        JSON_path_list = []
        for path in pathlist:
            path_str = str(path)
            JSON_path_list.append(path_str)
        return JSON_path_list
    
    def time_dict(season, date):
        file_path_list = JSON.pathlist(season, date)
        JSON_dict = dict()
        for file in file_path_list:
            path_metadata = glob.glob(f'{file}')
            metadata = str(path_metadata)[2:-2]
            with open(metadata) as f:
                meta = json.load(f)['lemnatec_measurement_metadata']
                time = (meta['gantry_system_variable_metadata']['time'])
                filename = os.path.basename(metadata)
            if JSON is not JSON_dict:
                JSON_dict[time, filename] = "Date, Time, and Image Name"
            else:
                print("JSON already in Dictionary")
        return sorted(JSON_dict)
    
    def time_df(season, date):
        JSON_time_d = JSON.time_dict(season, date)
        JSON_time_df = pd.DataFrame.from_dict(JSON_time_d)
        JSON_time_df.columns = ['Date and Time', 'Image Name']
        return JSON_time_df

In [49]:
EL = Thermal.retrieve_data('season_10_yr_2020', '2020-03-03', 10, 00)

In [6]:
JSON_time = JSON.time_df('season_10_yr_2020', '2020-03-03')

In [7]:
JSON_time['Date_Time']= pd.to_datetime(JSON_time['Date and Time'])

In [8]:
JSON_time['hour'] = JSON_time.Date_Time.dt.hour
JSON_time['minute'] = JSON_time.Date_Time.dt.minute
JSON_time['date'] = JSON_time.Date_Time.dt.date

In [9]:
JSON_time['time'] = pd.to_datetime(JSON_time['hour']*100 + JSON_time['minute'], format='%H%M').dt.time

In [10]:
round_JSON_time = JSON_time[['Image Name', 'Date_Time', 'date', 'time']]

In [40]:
round_JSON_time.set_index('time')

,Image Name,Date_Time,date
time,,,
08:45:00,fbf75978-ea6b-4d7c-8d85-9b21835c30fb_metadata....,2020-03-03 08:45:33,2020-03-03
08:45:00,4cb25050-63bf-4669-831f-2c6c7a5c31fa_metadata....,2020-03-03 08:45:35,2020-03-03
08:45:00,e26b63cf-265a-4235-9aed-1b0972c18fcf_metadata....,2020-03-03 08:45:37,2020-03-03
08:45:00,d71b8eef-4830-4839-94cb-a5d3e036684f_metadata....,2020-03-03 08:45:38,2020-03-03
08:45:00,f975830a-9676-49eb-a622-d2b05667ff6c_metadata....,2020-03-03 08:45:40,2020-03-03
...,...,...,...
13:27:00,cb43835e-f50a-47d9-b3ef-6304f4d34d09_metadata....,2020-03-03 13:27:46,2020-03-03
13:27:00,2141e0d5-7395-48d3-923b-eebe74586e1a_metadata....,2020-03-03 13:27:48,2020-03-03
13:27:00,958fa7a8-53de-400d-b7be-2a17bd6ceadc_metadata....,2020-03-03 13:27:50,2020-03-03


In [50]:
EL['time'] = EL.Time.dt.time

In [51]:
EL

,Time,Sun Direction,Temperature,Photosynthetically active radiation,time
0,2020-03-03 00:00:00,340.872219,9.833674,0.0,00:00:00
1,2020-03-03 00:01:00,341.344646,9.855037,0.0,00:01:00
2,2020-03-03 00:02:00,341.937925,9.821467,0.0,00:02:00
3,2020-03-03 00:03:00,342.454299,9.812311,0.0,00:03:00
4,2020-03-03 00:04:00,342.959685,9.781793,0.0,00:04:00
...,...,...,...,...,...
1435,2020-03-03 23:55:00,338.762780,12.070681,0.0,23:55:00
1436,2020-03-03 23:56:00,339.268166,12.070681,0.0,23:56:00
1437,2020-03-03 23:57:00,339.773553,12.073733,0.0,23:57:00
1438,2020-03-03 23:58:00,340.289926,12.058473,0.0,23:58:00


In [35]:
EL.set_index('time')
EL_df = pd.DataFrame(EL)

In [46]:
s = EL_df.loc[EL_df.index.unique()[round_JSON_time.index.unique().get_loc(dt.time, method='nearest')]]

TypeError: '<' not supported between instances of 'type' and 'int'